# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

## 설치
- `pip install gensim`

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문서의 리스트. 각 문서의 단어들을 리스트로 묶고 그 문서들을 리스트로 묶은 중첩 리스트.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

In [ ]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 단어와 vector를 매핑한 객체로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - "대상단어" in model.wv
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 **크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요**하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)